In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import NMF
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler,scale
# import cPickle

In [16]:
TRAINING_RATE = 0.5
TESTING_RATE = 1 - TRAINING_RATE
MISSING_RATE = 0.8
# QUERY_RATE = 0.2
np.random.seed(0)

In [17]:
data = pd.read_csv("../dat/wine_quality/winequality-white.csv",sep=';')

In [18]:
def preprocessing(d):
    #Add normalization code here if necessary
#     d.ix[:,0:-1] = scale(d.ix[:,0:-1])
    #0-1 scale
    min_max_scaler = MinMaxScaler()
    d.ix[:,0:-1] = min_max_scaler.fit_transform(d.ix[:,0:-1])

    d['quality'] = d['quality'].apply(lambda x: int(x) -3)
    d = d.iloc[np.random.permutation(len(d))]
    t = int(len(d) * TRAINING_RATE)
    tn_data = d.iloc[0:t,:]
    tt_data = d.iloc[t:,:]
    
    tn_X = tn_data.ix[:,0:-1]
    tn_Y = tn_data.ix[:,-1]
    tt_X = tt_data.ix[:,0:-1]
    tt_Y = tt_data.ix[:,-1]
    
#     return tn_X,tn_Y, tt_X, tt_Y
    return tt_X, tt_Y

In [19]:
def TestMissingGenerate(tt_data,mr=MISSING_RATE):
    missing_entry = []
    row , col= range(tt_data.shape[0]),range(tt_data.shape[1])
    
    while len(missing_entry) < tt_data.shape[0] * tt_data.shape[1] * mr:
        r = np.random.choice(row)
        c = np.random.choice(col)
        
        if (r,c) not  in missing_entry:
            missing_entry.append((r,c))
        else:
            continue
    
    for me in missing_entry:
        tt_data.set_value(me[0],tt_data.columns[me[1]],np.nan)
    return tt_data, missing_entry

In [20]:
def Imputation(d,missing):
    d = d.fillna(0.0)
    model = NMF(n_components=3, init='random', random_state=0)
    model.fit(d)
    H = model.components_
    W = model.fit_transform(d)
    d_prime = W.dot(H)
    
#     d = d.values
#     for m in missing:
#         d[m[0]][m[1]] = d_prime[m[0]][m[1]]
#         #d[m[0]][m[1]] = 0.0
    return d_prime

In [21]:
# tn_X,tn_Y,tt_X,tt_Y = preprocessing(data)
tt_X,tt_Y = preprocessing(data)

In [22]:
# tn_X = tn_X.reset_index(drop=True)
tt_X = tt_X.reset_index(drop=True)
tt_oracle = tt_X.copy()

In [23]:
tt_oracle.to_csv("../dat/wine_quality/wine_zeroone_oracle.csv",index=False,header=None)

In [24]:
tt_X, missing_entry = TestMissingGenerate(tt_X)

In [25]:
tt_X = tt_X.fillna(0)

In [26]:
tt_X.to_csv("../dat/wine_quality/wine_zeroone.csv",index=False,header=None)

In [27]:
tt_imp = Imputation(tt_X,missing_entry)

In [28]:
def calRMSE(result, oracle):
    return np.sum((result - oracle.values) ** 2)

In [29]:
calRMSE(tt_imp,tt_oracle)

1620.0523562448509

In [ ]:
#CASE1. NON_SCALE
#MF         #AF
17110999.61 5173595.76  #MISSING_RATE = 0.3
44086084.48 13897144.15 #MISSING_RATE = 0.8

#CALE2. [0,1]SCALE
#MF     #AF
558.36  309.47 #MISSING_RATE = 0.3
1620.05 333.09 #MISSING_RATE = 0.8
